In [48]:
import requests
import pandas as pd
import unicodedata
import time
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [50]:
def extract_demand(category='demanda', widget='evolucion', start_year=2011, end_year=2025):
    all_data = []

    for year in range(start_year, end_year):
        url = f"https://apidatos.ree.es/es/datos/{category}/{widget}"
        
        params = {'start_date': f'{year}-01-01T00:00',
              'end_date': f'{year}-12-31T23:59',
              'time_trunc': 'day'}


        headers = {'Accept': 'application/json',
                   'Content-Type': 'application/json',
                   'Host': 'apidatos.ree.es'}

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            all_data.append(data)
        else:
            print(f"Error fetching data for year {year}: {response.status_code}")

    demanda = []

    for entry in all_data:
        included = entry.get('included', [])
        for item in included:
            values = item.get('attributes', {}).get('values', [])
            for value in values:
                relevant = {'datetime': value.get('datetime'),
                            'demand_value': value.get('value'),
                            'percentage': value.get('percentage')}
                demanda.append(relevant)

    df_demanda = pd.DataFrame(demanda)
    df_demanda['fecha_extraccion'] = pd.Timestamp.now()
    df_demanda["fecha_extraccion"]= df_demanda["fecha_extraccion"].dt.floor("s")
    df_demanda.rename(columns={'datetime':'fecha', 'demand_value':'valor_demanda_MW', 'percentage':'porcentaje'},inplace=True)
    df_demanda.drop(['porcentaje'], axis=1, inplace=True)
    df_demanda['fecha']=df_demanda['fecha'].str.split('T').str[0]
    df_demanda['fecha']=pd.to_datetime(df_demanda['fecha'])
    return df_demanda


def extract_balance(start_year=2011, end_year=2025, time_trunc='day'):
    all_data = []

    for year in range(start_year, end_year):
        url = 'https://apidatos.ree.es/es/datos/balance/balance-electrico'
        
        params = {'start_date': f'{year}-01-01T00:00',
                  'end_date': f'{year}-12-31T23:59',
                  'time_trunc': time_trunc}

        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"Error fetching data for year {year}: {response.status_code}")
            continue
        
        balance_data = response.json()
        content_data = balance_data.get('included', [])[0].get('attributes', {}).get('content', [])

        data_list = []

        for item in content_data:
            type_name = item['type'] 
            values = item.get('attributes', {}).get('values', [])
            
            for value in values:
                value['type'] = type_name
                data_list.append(value)

        all_data.extend(data_list)  

    df_balance = pd.DataFrame(all_data) 
    df_balance['fecha_extraccion'] = pd.Timestamp.now()
    df_balance["fecha_extraccion"]= df_balance["fecha_extraccion"].dt.floor("s")
    df_balance.rename(columns={'datetime':'fecha', 'value':'valor_balance_GW', 'percentage':'porcentaje','type':'energia'},inplace=True)
    df_balance.drop(['porcentaje'], axis=1, inplace=True)
    df_balance['fecha']=df_balance['fecha'].str.split('T').str[0]
    df_balance['fecha']=pd.to_datetime(df_balance['fecha'])
    return df_balance


def extract_exchange(start_year=2011, end_year=2025, time_trunc='day', widget='todas-fronteras-fisicos'):
    all_lines = []

    for year in range(start_year, end_year):
        url = f'https://apidatos.ree.es/es/datos/intercambios/{widget}'
        
        params = {'start_date': f'{year}-01-01T00:00',
                  'end_date': f'{year}-12-31T23:59',
                  'time_trunc': time_trunc}

        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"Error fetching data for year {year}: {response.status_code}")
            continue  
        exchange_data = response.json()
        
        lines = []
        
        for country in exchange_data.get('included', []):
            country_name = country.get('id')
            
            if 'content' in country.get('attributes', {}): 
                for content in country['attributes']['content']:
                    trade_type = content.get('attributes', {}).get('title')
                    values = content.get('attributes', {}).get('values', [])
                    
                    for item in values:
                        line = {'country': country_name,
                                'type': trade_type,
                                'value': item.get('value'),
                                'percentage': item.get('percentage'),
                                'datetime': item.get('datetime')}
                        lines.append(line)
        
        all_lines.extend(lines)  

    df_exchanges = pd.DataFrame(all_lines) 
    df_exchanges['fecha_extraccion'] = pd.Timestamp.now()
    df_exchanges["fecha_extraccion"]= df_exchanges["fecha_extraccion"].dt.floor("s")
    df_exchanges.rename(columns={'datetime':'fecha', 'value':'valor_GW', 'percentage':'porcentaje','type':'tipo_transaccion','country':'pais'},inplace=True)
    df_exchanges.drop(['porcentaje'], axis=1, inplace=True)
    df_exchanges['fecha']=df_exchanges['fecha'].str.split('T').str[0]
    df_exchanges['fecha']=pd.to_datetime(df_exchanges['fecha'])
    return df_exchanges


def extract_generation(start_year=2011, end_year=2025, time_trunc='day'):
    all_gen_df = []

    for year in range(start_year, end_year):
        url = 'https://apidatos.ree.es/es/datos/generacion/estructura-generacion'
        
        params = {
            'start_date': f'{year}-01-01T00:00',
            'end_date': f'{year}-12-31T23:59',
            'time_trunc': time_trunc 
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"Error fetching data for year {year}: {response.status_code}")
            continue  
        
        generation_data = response.json()
        
        gen_df = []

        for included_data in generation_data.get('included', []):
            values = included_data.get('attributes', {}).get('values', [])
            
            df_gen = pd.DataFrame(values)
            
            df_gen['type'] = included_data.get('type')
            df_gen['id'] = included_data.get('id')
            df_gen['groupId'] = included_data.get('groupId')
            df_gen['title'] = included_data.get('attributes', {}).get('title')
            df_gen['description'] = included_data.get('attributes', {}).get('description')
            df_gen['color'] = included_data.get('attributes', {}).get('color')
            df_gen['technology_type'] = included_data.get('attributes', {}).get('type')
            
            gen_df.append(df_gen)

        all_gen_df.extend(gen_df)  

    df_generation = pd.concat(all_gen_df, ignore_index=True)  

    df_generation = df_generation[['datetime', 'value', 'percentage', 'type', 'id', 'groupId', 'title', 'description', 'color', 'technology_type']]
    df_generation['fecha_extraccion'] = pd.Timestamp.now()
    df_generation["fecha_extraccion"]= df_generation["fecha_extraccion"].dt.floor("s")
    df_generation.rename(columns={'datetime':'fecha', 'value':'valor_generacion_GW', 'percentage':'porcentaje','type':'energia', 'technology_type':'tipo_tecnología'},inplace=True)
    df_generation.drop(['porcentaje', 'title', 'groupId','id', 'description', 'color'], axis=1, inplace=True)
    df_generation['fecha']=df_generation['fecha'].str.split('T').str[0]
    df_generation['fecha']=pd.to_datetime(df_generation['fecha'])
    return df_generation

In [52]:
df_demanda=extract_demand()
df_balance=extract_balance()
df_exchanges=extract_exchange()
df_generation=extract_generation()

In [54]:
df_demanda

,fecha,valor_demanda_MW,fecha_extraccion
0,2011-01-01,605986.216,2024-10-31 14:19:24
1,2011-01-02,641856.087,2024-10-31 14:19:24
2,2011-01-03,801297.365,2024-10-31 14:19:24
3,2011-01-04,833253.263,2024-10-31 14:19:24
4,2011-01-05,803475.573,2024-10-31 14:19:24
...,...,...,...
5048,2024-10-27,582704.220,2024-10-31 14:19:24
5049,2024-10-28,665159.103,2024-10-31 14:19:24
5050,2024-10-29,697632.113,2024-10-31 14:19:24
5051,2024-10-30,683625.012,2024-10-31 14:19:24


In [56]:
df_balance

,valor_balance_GW,fecha,energia,fecha_extraccion
0,132922.642,2011-01-01,Hidráulica,2024-10-31 14:22:07
1,122475.160,2011-01-02,Hidráulica,2024-10-31 14:22:07
2,146861.638,2011-01-03,Hidráulica,2024-10-31 14:22:07
3,145876.053,2011-01-04,Hidráulica,2024-10-31 14:22:07
4,158946.134,2011-01-05,Hidráulica,2024-10-31 14:22:07
...,...,...,...,...
39139,316926.030,2024-10-27,Generación renovable,2024-10-31 14:22:07
39140,420991.631,2024-10-28,Generación renovable,2024-10-31 14:22:07
39141,432331.012,2024-10-29,Generación renovable,2024-10-31 14:22:07
39142,425839.989,2024-10-30,Generación renovable,2024-10-31 14:22:07


In [58]:
df_exchanges

,pais,tipo_transaccion,valor_GW,fecha,fecha_extraccion
0,Francia,Exportación,-3344.187,2011-01-01,2024-10-31 14:22:36
1,Francia,Exportación,-7900.880,2011-01-02,2024-10-31 14:22:36
2,Francia,Exportación,-13808.850,2011-01-03,2024-10-31 14:22:36
3,Francia,Exportación,-15848.206,2011-01-04,2024-10-31 14:22:36
4,Francia,Exportación,-19388.927,2011-01-05,2024-10-31 14:22:36
...,...,...,...,...,...
60259,Andorra,saldo,-1183.610,2024-09-26,2024-10-31 14:22:36
60260,Andorra,saldo,-816.820,2024-09-27,2024-10-31 14:22:36
60261,Andorra,saldo,-326.360,2024-09-28,2024-10-31 14:22:36
60262,Andorra,saldo,-318.320,2024-09-29,2024-10-31 14:22:36


In [60]:
df_generation

,fecha,valor_generacion_GW,energia,tipo_tecnología,fecha_extraccion
0,2011-01-01,132922.642,Hidráulica,Renovable,2024-10-31 14:22:45
1,2011-01-02,122475.160,Hidráulica,Renovable,2024-10-31 14:22:45
2,2011-01-03,146861.638,Hidráulica,Renovable,2024-10-31 14:22:45
3,2011-01-04,145876.053,Hidráulica,Renovable,2024-10-31 14:22:45
4,2011-01-05,158946.134,Hidráulica,Renovable,2024-10-31 14:22:45
...,...,...,...,...,...
86063,2024-10-27,613732.920,Generación total,Generación total,2024-10-31 14:22:45
86064,2024-10-28,724283.903,Generación total,Generación total,2024-10-31 14:22:45
86065,2024-10-29,740930.113,Generación total,Generación total,2024-10-31 14:22:45
86066,2024-10-30,744731.012,Generación total,Generación total,2024-10-31 14:22:45


In [ ]:
host= 
user= 
password= 
database= 

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

def insert_data(df_demanda, df_balance, df_exchanges, df_generation):
    with engine.connect() as connection:
        # Insertar dataframes en tablas de la base de datos
        df_demanda.to_sql('demanda_energia', con=connection, if_exists='replace', index=False)
        df_balance.to_sql('balance_energia', con=connection, if_exists='replace', index=False)
        df_exchanges.to_sql('transacciones_energia', con=connection, if_exists='replace', index=False)
        df_generation.to_sql('generacion_energia', con=connection, if_exists='replace', index=False)

In [ ]:
insert_data(df_demanda, df_balance, df_exchanges, df_generation)